 # Luonnollisen kielen käsittelyn (NLP) demo

## Lähteitä

Koodia, esimerkkejä:

- Demo mukailee esimerkkitutorialia: https://www.tensorflow.org/hub/tutorials/tf2_text_classification
- Osia myös tutorialista: https://medium.com/intro-to-artificial-intelligence/entity-extraction-using-deep-learning-8014acac6bb8
- ...ja täältä: https://medium.com/swlh/using-xlnet-for-sentiment-classification-cfa948e65e85-
- ...ja täältä: https://github.com/kcmankar/pytorch-sentiment-analysis-using-XLNet/blob/master/xlnet_sentiment_analysis.ipynb
- .. ja täältä: https://news.machinelearning.sg/posts/sentiment_analysis_on_movie_reviews_with_xlnet/

Dataa: 

- Sentimenttidata: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb
- Sentimenttidata, alkuperäinen lähde? : https://ai.stanford.edu/~amaas/data/sentiment/
- IMDb-data: https://www.imdb.com/interfaces/

Menetelmiä:

- XLNet: https://github.com/zihangdai/xlnet
- XLNet-paperi: https://arxiv.org/pdf/1906.08237.pdf
- GLUE-sivusto: https://gluebenchmark.com/
- GLUE-paperi: https://openreview.net/pdf?id=rJ4km2R5t7

Muita lähteitä:

- NLP-kehitystä seuraileva GitHub-repo: https://github.com/sebastianruder/NLP-progress
- Toinen NLP-GitHub -repo: https://github.com/keon/awesome-nlp
- Turku NLP: https://turkunlp.org/
- Kieliriippumattomia sana-assosiaatioita https://universaldependencies.org/

## Demo

Määritellään tarvittavat Python-kieliset paketit:

In [ ]:
import os
import pickle
import numpy as np

# NNLP:tä varten:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Kytketään pois GPU:n puuttumisesta kertovat virheviestit.

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow import keras

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# XLNetiä varten:

#  import transformers
# from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
# import torch

# import pandas as pd
# import seaborn as sns

# from matplotlib import rc
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score # accuracy vai accuracy_score?
# from sklearn.utils import shuffle
# from collections import defaultdict
# from textwrap import wrap
#from pylab import rcParams

# from torch import nn, optim
# from keras.preprocessing.sequence import pad_sequences
# from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
# from torch.utils.data import Dataset, DataLoader
# import torch.nn.functional as F

# import re

In [ ]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [ ]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

In [ ]:
train_examples[0]

In [ ]:
train_labels[0]

In [ ]:
train_examples[5]

In [ ]:
train_labels[5]

model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
# hub_layer(train_examples[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

![title](neural_net.png)

Lähde https://www.cs.mcgill.ca/~jcheung/teaching/fall-2016/comp599/lectures/lecture23.pdf "Goldberg et al. 2015")

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
train_cutoff = 10000

x_val = train_examples[:train_cutoff]
partial_x_train = train_examples[train_cutoff:]

y_val = train_labels[:train_cutoff]
partial_y_train = train_labels[train_cutoff:]

Mallin opetusta (kesto olisi n. 12 min)

In [ ]:
model_dir = '../work/nlp_model'

if os.path.exists(model_dir):
    model = keras.models.load_model(model_dir)

    with open(os.path.join(model_dir, 'hist.pkl'), 'rb') as handle:
        hist = pickle.load(handle)
else:
    history = model.fit(partial_x_train,
                        partial_y_train,
                        epochs=40,
                        batch_size=512,
                        validation_data=(x_val, y_val),
                        verbose=1)
    model.save(model_dir)
                                
    with open(os.path.join(model_dir, 'hist.pkl'), 'wb') as handle:
        pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    hist = history.history

In [ ]:
results = model.evaluate(test_data, test_labels)

In [ ]:
history_dict = hist
history_dict.keys()

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
path_to_data = '../data/IMDb_kaggle/IMDB Dataset.csv'
df = pd.read_csv(path_to_data)

# Shuffle and Clip data
df = shuffle(df)
df = df[:24000]

# Function to clean text. Remove tagged entities, hyperlinks, emojis
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
 
df['review'] = df['review'].apply(clean_text)

# Function to convert labels to number.
def sentiment2label(sentiment):
    if sentiment == "positive":
        return 1
    else :
        return 0

df['sentiment'] = df['sentiment'].apply(sentiment2label)

# List of class names.
class_names = ['negative', 'positive']

In [ ]:
from transformers import XLNetTokenizer, XLNetModel
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class ImdbDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.5, random_state=101)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = ImdbDataset(
    reviews=df.review.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
EPOCHS = 3
BATCH_SIZE = 4
MAX_LEN = 512

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

In [ ]:
EPOCHS = 3

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [ ]:
next(iter(val_data_loader))

In [ ]:
data = next(iter(val_data_loader))
data.keys()

In [ ]:
# Versio 2

In [ ]:
!pip install pytorch-transformers

In [ ]:
path_to_data = '../data/sentiment/amazon_cells_labelled.txt'
fd = pd.read_csv(path_to_data, sep='\t')
fd.columns = ['sentence','value']

In [ ]:
sentences  = []
for sentence in fd['sentence']:
  sentence = sentence+"[SEP] [CLS]"
  sentences.append(sentence)

In [ ]:
sentences[0]

In [ ]:
from pytorch_transformers import XLNetTokenizer,XLNetForSequenceClassification

In [ ]:
from sklearn.model_selection import train_test_split
from pytorch_transformers import AdamW
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

In [ ]:
tokenizer  = XLNetTokenizer.from_pretrained('xlnet-base-cased',do_lower_case=True)
tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
tokenized_text[0]

In [ ]:
ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]

In [ ]:

print(ids[0])
labels = fd['value'].values
print(labels[0])

In [ ]:
max1 = len(ids[0])
for i in ids:
  if(len(i)>max1):
    max1=len(i)
print(max1)
MAX_LEN = max1

In [ ]:
input_ids2 = pad_sequences(ids,maxlen=MAX_LEN,dtype="long",truncating="post",padding="post")

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(input_ids2,labels,test_size=0.15)

In [ ]:
print(len(input_ids2[0]))

In [ ]:
Xtrain = torch.tensor(xtrain)
Ytrain = torch.tensor(ytrain)
Xtest = torch.tensor(xtest)
Ytest = torch.tensor(ytest)

In [ ]:
batch_size = 3

In [ ]:
train_data = TensorDataset(Xtrain,Ytrain)
test_data = TensorDataset(Xtest,Ytest)
loader = DataLoader(train_data,batch_size=batch_size)
test_loader = DataLoader(test_data,batch_size=batch_size)

In [ ]:
%tensorflow_version 1.x

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",num_labels=2)

In [ ]:
optimizer = AdamW(model.parameters(),lr=2e-5)

In [ ]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [ ]:
import numpy as np
def flat_accuracy(preds,labels):  # A function to predict Accuracy
  correct=0
  for i in range(0,len(labels)):
    if(preds[i]==labels[i]):
      correct+=1
  return (correct/len(labels))*100

In [ ]:
no_train = 0
epochs = 3
for epoch in range(epochs):
  model.train()
  loss1 = []
  steps = 0
  train_loss = []
  l = []
  for inputs,labels1 in loader :
    inputs.to(device)
    labels1.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(device))
    loss = criterion(outputs[0],labels1.to(device)).to(device)
    logits = outputs[1]
    #ll=outp(loss)
    [train_loss.append(p.item()) for p in torch.argmax(outputs[0],axis=1).flatten() ]#our predicted 
    [l.append(z.item()) for z in labels1]# real labels
    loss.backward()
    optimizer.step()
    loss1.append(loss.item())
    no_train += inputs.size(0)
    steps += 1
  print("Current Loss is : {} Step is : {} number of Example : {} Accuracy : {}".format(loss.item(),epoch,no_train,flat_accuracy(train_loss,l)))

In [ ]:
model.eval()#Testing our Model
acc = []
lab = []
t = 0
for inp,lab1 in test_loader:
  inp.to(device)
  lab1.to(device)
  t+=lab1.size(0)
  outp1 = model(inp.to(device))
  [acc.append(p1.item()) for p1 in torch.argmax(outp1[0],axis=1).flatten() ]
  [lab.append(z1.item()) for z1 in lab1]
print("Total Examples : {} Accuracy {}".format(t,flat_accuracy(acc,lab)))

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers
!pip install -q datasets

In [ ]:
import pandas as pd
from datasets import load_dataset

dataset_train = load_dataset('imdb',split='train')
dataset_train.rename_column_('label', 'labels')
train_df=pd.DataFrame(dataset_train)

dataset_test = load_dataset('imdb',split='test')
dataset_test.rename_column_('label', 'labels')
test_df=pd.DataFrame(dataset_test)

In [ ]:
train_df.head()

In [ ]:
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]], 
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
# Prints out the dataset sizes of train test and validate as per the table.
pd.DataFrame(data, columns=["Train", "Test"])

In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'learning_rate': 0.00001,
    'weight_decay': 0.01,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '../work/xlnet_outputs/',
}

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=2, args=train_args, use_cuda=False) 

# Train the model, there is no development or validation set for this dataset 
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)